# Matrix Encryption
Makenna Worley and Nic Van Der Werf

An exploration into encrypting data through matrices transformations and modular arithmetic

In [20]:
# Importing and declaring global variables
import numpy as np
import math
import random

from ipywidgets import widgets
from IPython.display import display

MODULO = 59
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?'\n"
# print(len(ALPHABET))

KEY_MATRIX_2x2 = np.array([
    [3, 2],
    [5, 7]
])

text1 = open('text/one.txt').read()
text2 = open('text/two.txt').read()
text3 = open('text/our_super_secret_message.txt').read()

Functions that will help us with encrypting and decrypting our secret messages!

Generates a random 2 x 2 Key Matrix. Checks to make sure Greatest Common Denominator is 1 and that it is invertible with the modulo.

In [2]:
def generate_key_matrix_2x2(modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(2, 2))

        # Calculating the determinant with Mod
        a, b = key_matrix[0]
        c, d = key_matrix[1]
        det = (a*d - b*c) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")

Inverts a 2 x 2 Matrix with the modulo using integers 

In [3]:
def inverse_2x2_mod(matrix, modulo):
    a, b = matrix[0]
    c, d = matrix[1]
    det = int((a*d - b*c) % modulo)
    if math.gcd(det, modulo) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(modulo))
    det_inv = pow(det, -1, modulo)
    return (det_inv * np.array([[d, -b],
                                [-c, a]])) % modulo

Encrypts plain text using a 2 x 2 key matrix. by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

In [4]:
def encrypt(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    plaintext_matrix = np.array(numbers).reshape(-1, 2).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix

Decrypts plain text using a 2 x 2 inverse key matrix by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the inverse key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

In [5]:
def decrypt(ciphertext, inverse_key_matrix, modulo, alphabet):
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    cipher_matrix = np.array(numbers).reshape(-1, 2)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('*')
    return ''.join(plaintext_chars).rstrip('A')

Generates a random n x n Key Matrix. Checks to make sure Greatest Common Denominator is 1 and that it is invertible with the modulo.


In [6]:
def generate_key_matrix_nxn(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        det = int(round(np.linalg.det(key_matrix))) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_nxn_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            # All matrices must be invertible
            pass

Inverts a n x n matrix with the modulo using Gaussian Elimination

Inverting matrices with >2 rows and columns requires GCD since it is not guaranteed that rows and columns will be the same number, so we need to look at the coprime of them

In [7]:
# Inverting matrices with >2 rows and columns requires GCD since it is not guaranteed that rows and columns will be the same number, so we need to look at the coprime of them
def inverse_nxn_mod(matrix, modulo):
    A = np.array(matrix, dtype=int) % modulo
    n = A.shape[0]

    I = np.eye(n, dtype=int)

    # Perform Gaussian elimination
    for i in range(n):
        # Find pivot (row with non-zero element in column i)
        for r in range(i, n):
            if A[r, i] % modulo != 0:
                # Swap pivot row if necessary
                if r != i:
                    A[[i, r]] = A[[r, i]]
                    I[[i, r]] = I[[r, i]]
                break
        else:
            # No pivot found, matrix not invertible
            raise ValueError("Matrix not invertible modulo {}".format(modulo))

        # Normalize pivot row
        inv_pivot = pow(int(A[i, i]), -1, modulo)
        A[i] = (A[i] * inv_pivot) % modulo
        I[i] = (I[i] * inv_pivot) % modulo

        # Eliminate other rows
        for r in range(n):
            if r != i:
                factor = A[r, i] % modulo
                if factor != 0:
                    A[r] = (A[r] - factor * A[i]) % modulo
                    I[r] = (I[r] - factor * I[i]) % modulo

    return I % modulo

Encrypts plain text using a n x n key matrix. by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

Matrix created from text is dynamically sized so that the dot product can be taken with the n x n key


In [8]:
def encrypt_nxn(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    n = key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    plaintext_matrix = np.array(numbers).reshape(-1, n).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix

Decrypts plain text using a 2 x 2 inverse key matrix by converting text to integers based on the provided alphabet, creating a matrix of those integers, getting the dot product with the inverse key, then converting back to text. Modulo is used to ensure all digits are in the range of the alphabet.

Matrix created from text is dynamically sized so that the dot product can be taken with the n x n inverse key


In [9]:
def decrypt_nxn(ciphertext, inverse_key_matrix, modulo, alphabet):
    n = inverse_key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    cipher_matrix = np.array(numbers).reshape(-1, n)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    dec = ''.join(plaintext_chars)
    return dec.rstrip('A')

Functions for creating and testing random strings with a 2 x 2 key matrix

In [10]:
def generate_random_string(alphabet, l):
    return ''.join(random.choice(alphabet) for _ in range(l))


def test_encryption_decryption(l):
    plaintext = generate_random_string(ALPHABET, l)
    ciphertext, ciphertext_matrix = encrypt(plaintext, KEY_MATRIX_2x2, MODULO, ALPHABET)
    decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
    return plaintext, decrypted_text, plaintext == decrypted_text

First testing our matrix to make sure it is invertible and therefore useable for our purposes.

In [16]:
inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)

for idx in range(1, 51):
    plaintext, decrypted_text, result = test_encryption_decryption(200)
    if result:
        print(f"Test {idx}: True")
    else:
        print(f"Test {idx}: False")
        print(f"  Original: {plaintext}")
        print(f"  Decrypted: {decrypted_text}")

Test 1: True
Test 2: True
Test 3: True
Test 4: True
Test 5: True
Test 6: True
Test 7: True
Test 8: True
Test 9: True
Test 10: True
Test 11: True
Test 12: True
Test 13: True
Test 14: True
Test 15: True
Test 16: True
Test 17: True
Test 18: True
Test 19: True
Test 20: True
Test 21: True
Test 22: True
Test 23: True
Test 24: True
Test 25: True
Test 26: True
Test 27: True
Test 28: True
Test 29: True
Test 30: True
Test 31: True
Test 32: True
Test 33: True
Test 34: True
Test 35: True
Test 36: True
Test 37: True
Test 38: True
Test 39: True
Test 40: True
Test 41: True
Test 42: True
Test 43: True
Test 44: True
Test 45: True
Test 46: True
Test 47: True
Test 48: True
Test 49: True
Test 50: True


Now let's look at our encrypting a message using our predefined KEY_MATRIX_2x2

In [17]:
MESSAGE = "Hello my name is Makenna"

inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)
print(f"Key Matrix:\n{KEY_MATRIX_2x2}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, KEY_MATRIX_2x2, MODULO, ALPHABET)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")
print(f"Condition: {np.linalg.cond(KEY_MATRIX_2x2)}")

original_text = MESSAGE

print("Cipher Text Matrx: \n", ciphertext_matrix)
print("\n\n")

print("Original Text :   ",original_text)
print()
print("Cipher Text   :   ",ciphertext)
print()
print("Decrypted Text:   ",decrypted_text)


Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[ 6 32]
 [21 11]]
Ciphertext Matrix:
[[22  9]
 [ 8 31]
 [47 33]
 [37  9]
 [57  2]
 [36 42]
 [17 42]
 [13  6]
 [ 3 49]
 [32 28]
 [50 10]
 [51 23]]
Ciphertext: WJIfvhlJ'CkqRqNGDxgcyKzX
Condition: 7.780565547548909
Cipher Text Matrx: 
 [[22  9]
 [ 8 31]
 [47 33]
 [37  9]
 [57  2]
 [36 42]
 [17 42]
 [13  6]
 [ 3 49]
 [32 28]
 [50 10]
 [51 23]]



Original Text :    Hello my name is Makenna

Cipher Text   :    WJIfvhlJ'CkqRqNGDxgcyKzX

Decrypted Text:    Hello my name is Makenna


Example with the generating of a 2 x 2 random key. 

In [18]:
MESSAGE = "Hello my name is Nic"

key_matrix, inverse_key_matrix = generate_key_matrix_2x2(MODULO)

print(f"Key Matrix:\n{KEY_MATRIX_2x2}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, key_matrix, MODULO, ALPHABET)

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)

original_text = MESSAGE

print("Cipher Text Matrx: \n", ciphertext_matrix)
print("\n\n")

print("Original Text :   ",original_text)
print()
print("Cipher Text   :   ",ciphertext)
print()
print("Decrypted Text:   ",decrypted_text)

Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[26 20]
 [21  7]]
Cipher Text Matrx: 
 [[10  8]
 [34 55]
 [26 39]
 [13 44]
 [31 39]
 [53 15]
 [ 2 52]
 [26 21]
 [ 7 23]
 [43 52]]



Original Text :    Hello my name is Nic

Cipher Text   :    KIi!anNsfn.PC aVHXr 

Decrypted Text:    Hello my name is Nic


In [22]:
MESSAGE = text1
KEY_MATRIX_SIZE = 7

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Plain Text Length: 93
Ciphertext Length: 98
Key Matrix:
[[ 2 22  3 39  3  8  7]
 [57 43 36 41  4  9 44]
 [38 25 47 21 47 29 13]
 [20 15  9 54  7 12 24]
 [19 39 25 19 49 46 49]
 [53 15 23 26 55 58 19]
 [23 50 48 32 58 15 27]]
Inverse Key Matrix:
[[36 22 26 54 43 24 31]
 [54 57 41 39 44 30 24]
 [20 43 11 21 50 18 29]
 [11 44 45 16 29 23 48]
 [21 33 28 23 46  2 23]
 [17 29 33  6 25 16 30]
 [ 0  1 39  0 48 22 30]]
Ciphertext: CCztzyPvgxwkreVSoNriB.RrfqS'nmVIQ,nbzJulGLAEGfKV?GuadVsaA!SJqipcPdKJyuI,s.r ZAccQEdnvLPoI.gZD'VsI



Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()

In [23]:
MESSAGE = text2
KEY_MATRIX_SIZE = 23

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Plain Text Length: 93
Ciphertext Length: 69
Key Matrix:
[[56 39 13 39 26 36 42  3 50 35 10 48 45 17 39 20 44 36 58 56 29  9 42]
 [35 51  0 30 44 54 36 23  5 51 37 14 47 19 27 51 17 19 44 28 17 11  8]
 [27  4  4 47 33 11 42 57  9 16 42 32 51  1 48 45 18 18 36 32  8 44 19]
 [34 42 48 38 15  4 48 11 40 50 28 50 55 35 31 41 16 30 25 16 50 44 33]
 [12  2 40 13 30  2  1  3 42 15 16 14 35 34 22  4 37 48 30  4  3 55 33]
 [47 49 36 15 28 15  8 32 43 58 17 55  5 45 49 15 39 30 30 31 29  1  4]
 [32 43 46 34  9  2 35 39 11 42 20 15 43 42 29 54  9 33 24  9 54 58 25]
 [ 9  7 41 58 22  9 36 53 57  4 27 43 31 44  5 45 49 40 37 51  4 22 54]
 [47 57 54 46 34 35  2 26 29 10 35 25  5 34 53 18  1 39 24  9 33  1 35]
 [54 17 31 27 19 55 29  4 34 42 25 12 10 45 26 36  4 25 56 27 32 27 22]
 [ 5 27  3 18 39 32 35 44 46 16 12 53 51 39 43  5 23 36 29  6  7 26 13]
 [52 28 15  1 28 48  3 53 46 52 53 46  4  7 35 24  3 27 43 46 37 37 33]
 [29 25 16 55 37 33  7 17 37 23 23  3 12 39 20 13 52 26 53  4 12 43 32]
 [ 3 53 

Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()

In [24]:
MESSAGE = text3
KEY_MATRIX_SIZE = 100

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Plain Text Length: 93
Ciphertext Length: 1800
Key Matrix:
[[35 56  4 ... 56 16  1]
 [43 47 56 ... 22  9 44]
 [54 24 30 ... 40 31 34]
 ...
 [47 28 29 ... 29  5 47]
 [ 2  4  7 ...  2 35 53]
 [23 54 10 ... 44  4 40]]
Inverse Key Matrix:
[[29 35 53 ...  6 24 49]
 [36 25 52 ... 50 57 39]
 [ 2 52 45 ...  4 15  7]
 ...
 [53 14 13 ...  3 27 37]
 [36 22  6 ... 36 12  5]
 [47 44  2 ... 10 16 55]]
Ciphertext: QVYBAJRuLcD'Vq,wgawsTevbL,o?,nFpqZEuGOmjJipoZZzqbiVexry.tbIukHul'DoC ,koiSoABDJ hBVcddCtOuEiLL.D?xli'wM. U VOuLA'bkWIvgQ!QaVa.Lvc'nv!HHcZhDKkqEPWeWZPW.eHXfSJNtN'mlFGZWHPKpEfbIDfR'rcSLfEK.H.D,mvKysCpqnKMy'gV'BKrn
 DQA?aOAvqAcTLSPBPcjB
RLEN.nxaLQ?pE.DADzly.ItLgmajSrs?tiU'Sng
itvPkVTsUMrdEUKmlkPiFDwwgAxnRqC aXBfSvlIkvVXtJGm
CMbqri
VXcgBCT
DGm
XKITq?fopb.qEPXSPHkn?TMiakktQVHAlRWed!DTzbnwazigQLPTc?E V?plLSvcKFwTqAGCvEhZ!xjOJ,MOouzUIR??
OrWAjm
lLGo GNsdgCGKDMmgJnUlEHfBF.oijygxitdTLJyc'BAtpLeNOfnOxDCUBRIHa'fPLWTWOcAkEyBWrSAkZcF!xfd
.h'H?xDXJLeGDWpJXbwtPTps
,iUE.nbHeeRlGX?NXhKQvLWjbp pJaCcdSMYflvHGQ

Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()